# Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:
| **`Tracking server`**   | yes local server |
| :---------------------: | :--------------: | 
| **`Backend store`**     | sqlite database  |
| **`Artifact store`**    | local filesystem |

The experiment can be explored locally by accessing the local tracing server.

To run this example you need to launch the `mlflow` server locally by running the following command in your terminal:

```shell
mlflow server --backend-store-uri sqlite:///backend.db
```

You can also add the parameter `--default-artifact-root <path>`

In [2]:
import mlflow

# Connect after running the following command:
# mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking-URI: '{mlflow.get_tracking_uri()}'")

tracking-URI: 'http://127.0.0.1:5000'


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1686316294925, experiment_id='0', last_update_time=1686316294925, lifecycle_stage='active', name='Default', tags={}>]

**Code for training a Logistic Regression Model**

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    X, y = load_iris(return_X_y=True)
    params = { "C": 0.1, "random_state": 42 }
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")


2023/06/09 15:15:54 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: '/media/userl/Ubuntu-DataStora/Learning/MLOps_Zoomcamp/mlops-material/2-Experiment-Tracking/running-mlflow-examples/artifacts_local/1/1e076a52ad254826a4d8d7b7daf6ab88/artifacts'


/home/userl/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Metadata is now stored in the SQLite database instead of the local file system.

In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/media/userl/Ubuntu-DataStora/Learning/MLOps_Zoomcamp/mlops-material/2-Experiment-Tracking/running-mlflow-examples/artifacts_local/1', creation_time=1686316554787, experiment_id='1', last_update_time=1686316554787, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1686316294925, experiment_id='0', last_update_time=1686316294925, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the Model Registry

In [9]:
from mlflow.tracking import MlflowClient
client = MlflowClient("http://127.0.0.1:5000")

In [10]:
# There are currently no registered models
client.search_registered_models()

[]

In [24]:
# Register a model
run_id = client.search_runs(experiment_ids="1")[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models", 
    name="iris-classifier"
)

Successfully registered model 'iris-classifier'.
2023/06/09 15:28:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1686317313224, current_stage='None', description='', last_updated_timestamp=1686317313224, name='iris-classifier', run_id='1e076a52ad254826a4d8d7b7daf6ab88', run_link='', source='/media/userl/Ubuntu-DataStora/Learning/MLOps_Zoomcamp/mlops-material/2-Experiment-Tracking/running-mlflow-examples/artifacts_local/1/1e076a52ad254826a4d8d7b7daf6ab88/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

Now look into the MLflow model registry. The model will be there.